<a href="https://colab.research.google.com/github/rafaelbkneip/reclassificacoes-USP-SISU/blob/main/Reclassifica%C3%A7%C3%B5es_USP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalações / Installs
! pip install pdfplumber -q
! pip install PyPDF2

#Importações / Imports
import pdfplumber
from google.colab import files
from PyPDF2 import PdfReader, PdfWriter

In [ ]:
#Upload do arquivo gerado pelo USP / Upload file generated by USP
pdf = files.upload()

#Abrir e ler o arquivo / Open and read file
pdf = pdfplumber.open('')
readpdf = PdfReader('')


In [ ]:
#Lista auxiliar / Auxiality list
unica = []

#Extratir o texto de cada página / Extract he text of each page 
for j in range(len(readpdf.pages)):
  text = pdf.pages[j].extract_text()
  listas = text.split("\n")

  #Para cada página, obter apenas as informações necessárias / For each page, get only the necessary informations
  for i in range(len(listas)):
    if(listas[i].split(" ")[0]!= "ENEM" and listas[i]!= "CHAMADOS PARA A SEGUNDA MATRÍCULA" and listas[i]!= "PROCESSAMENTO REALIZADO EM 28/02/2023 21:03:46"):
      #Criar uma lista unica com a informação de cada página / Create a single list with all the information of each page
      unica.append(listas[i])

print(unica)

In [ ]:
#Lista para os alunos / List for the studentes
alunos = []
#Lista auxiliar / Auxialiary list
aux = []

#Separar o texto de linha uma linha em elementos de uma lista / Split the one-line text into element lists
for r in range(len(unica)-1):
  #As linhas com informação dos alunos, encontram-se entre linhas com infomação do curso e campus / Lines with student information are between lines with course and campus information
  if(unica[r].split(" ")[0].isnumeric() == False and unica[r+1].split(" ")[0].isnumeric() == False):
    curso = unica[r]
    faculdade = unica[r+1]

    for k in range(r+2, len(unica)-1):
      if(unica[k].split(" ")[0].isnumeric()):
          aux.append(unica[k].split(" ")[0])

          nome = ""
          for p in range(2, len(unica[k].split(" "))):
            nome = nome  + unica[k].split(" ")[p] + " "

          aux.append(nome)
          aux.append(curso)
          aux.append(faculdade)
          
          alunos.append(aux)

          aux = []

      else:
        break

#Último aluno da listagem / Last student on the list
aux = []
aux.append(unica[-1].split(" ")[0])

nome = ""

for p in range(2, len(unica[-1].split(" "))):
    nome = nome  + unica[-1].split(" ")[p] + " "

aux.append(nome)
aux.append(curso)
aux.append(faculdade)
alunos.append(aux)

print(alunos)

In [ ]:
#Importações / Imports
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
drive.mount('/content/gdrive')

creds, _ = default()
gc = gspread.authorize(creds)
planilha = gc.open('USP - 2ª chamada')

In [ ]:
from time import sleep

#Escolher a planilha / Select the sheet
pagina = planilha.get_worksheet(0)

#Escrever as informações presentes listas na planilha / Write the information in lists on the Google Spreadsheet spreadsheet
for j in range(len(alunos)-1, len(alunos)):
    lista_cel = pagina.range('A' + str(j+1) +':' + 'D' +  str(j+1))
    for i in range (len(alunos[0])):
        lista_cel[i].value = alunos[j][i]

    sleep(3)

    pagina.update_cells(lista_cel, value_input_option="USER_ENTERED")